In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize

In [2]:
def preprocess_dataframe(df):
    # print(df.shape[0])
    df['raw_pred'].fillna("No answer", inplace=True)
    df['raw_pred'] = [x.replace("\\n", "") for x in df['raw_pred']]
    df['raw_pred'] = [x[:x.find('### USER:')] if '### USER:' in x else x for x in df['raw_pred']]
    df['Explanation for correct answer'].fillna("", inplace=True)
    df['Explanation for correct answer'] = [x.replace("Explanation: ", "") for x in df['Explanation for correct answer']]
    df['raw_pred'] = [x.replace("Explanation: ", "") for x in df['raw_pred']]
    df['raw_pred'] = [x.strip() for x in df['raw_pred']]
    df['raw_pred'] = [x.replace("\n", " ") for x in df['raw_pred']]
    # df['raw_pred'] = [x[:x.find("\n") + 1] for x in df['raw_pred']]
    # print(df.shape[0])

    raw_preds = []
    true_preds = []
    for idx, row in enumerate(df.iterrows()):
        sentences_raw = sent_tokenize(row[1]['raw_pred'])
        
        sentences_true = sent_tokenize(row[1]['Explanation for correct answer'])
        if len(sentences_true) == 0:
            sentences_true.append('')
        if len(sentences_raw) == 0:
            sentences_raw.append('')
        sentences_raw = [x.replace(".", "") for x in sentences_raw]
        sentences_raw = [x.replace("(A", "(A)") for x in sentences_raw]
        sentences_raw = [x.replace("(B", "(B)") for x in sentences_raw]
        sentences_raw = [x.replace("(C", "(C)") for x in sentences_raw]
        sentences_raw = [x.replace("(D", "(D)") for x in sentences_raw]
        sentences_raw = [x.replace("))", ")") for x in sentences_raw]
        sentences_true = [x.replace(".", "") for x in sentences_true]

        if '(A)' in sentences_raw[0] or '(B)' in sentences_raw[0] or '(C)' in sentences_raw[0] or '(D)' in sentences_raw[0]:
            sentences_raw = sentences_raw[1: ] + [sentences_raw[0]]

        if '(A)' not in sentences_raw[-1] and '(B)' not in sentences_raw[-1] and '(C)' not in sentences_raw[-1] and '(D)' not in sentences_raw[-1]:
            pass
            # print(idx)
            # print('. '.join(sentences_raw))
        try:
            if '(A)' in sentences_true[0] or '(B)' in sentences_true[0] or '(C)' in sentences_true[0] or '(D)' in sentences_true[0]:
                sentences_true = sentences_true[1: ] + [sentences_true[0]]
        except Exception as e:
            print(idx)
            print(sentences_true)
            pass
        try:
            if '(A)' not in sentences_true[-1] and '(B)' not in sentences_true[-1] and '(C)' not in sentences_true[-1] and '(D)' not in sentences_true[-1]:
                sentences_true = sentences_true + [f"Therefore, the correct answer is ({row[1]['correct_answer_letter']})"]
        except Exception as e:
            print(sentences_true)
            pass
        raw_preds.append('. '.join(sentences_raw))
        true_preds.append('. '.join(sentences_true))
    df['raw_pred'] = raw_preds
    df['Explanation for correct answer'] = true_preds
    return df  


In [3]:
import os
os.listdir('./')

['BioMedGPT-LM-7B_few-shot-cot_None_3.csv',
 'Mistral-7B-Instruct-v0.3_few-shot-cot_None_1.csv',
 'gsm8k.csv',
 'meditron-70b_few-shot-cot_None_1.csv',
 'Asclepius-7B_few-shot-cot_None_3.csv',
 'ClinicalCamel-70B_few-shot-cot_None_3.csv',
 'drop.csv',
 'Meta-Llama-3-8B-Instruct_few-shot-cot_None_1.csv',
 'Llama-2-70b-hf_few-shot-cot_None_1.csv',
 'ClinicalCamel-70B_few-shot-cot_None_1.csv',
 'MentaLLaMA-chat-13B_few-shot-cot_None_3.csv',
 'llama-2-7b-chat-hf_few-shot-cot_None_1.csv',
 'Llama3-OpenBioLLM-8B_few-shot-cot_None_3.csv',
 'Llama3-Med42-70B_few-shot-cot_None_1.csv',
 'Meta-Llama-3-70B-Instruct_few-shot-cot_None_1.csv',
 'Llama-2-70b-hf_few-shot-cot_None_3.csv',
 'cosmos.csv',
 'Llama-2-13b-hf_few-shot-cot_None_1.csv',
 'qa_csv_to_json.ipynb',
 'medalpaca-13b_few-shot-cot_None_3.csv',
 'medalpaca-7b_few-shot-cot_None_1.csv',
 'MentaLLaMA-chat-13B_few-shot-cot_None_1.csv',
 'Asclepius-13B_few-shot-cot_None_3.csv',
 'Llama-2-13b-chat-hf_few-shot-cot_None_1.csv',
 'counselingQA-g

In [4]:
for file in os.listdir('./'):
    if 'few-shot-cot_None_3' in file:
        model = file[:file.find('_')]
        model = model.replace('-', '_')
        df = pd.read_csv(os.path.join('.', file))
        print(df.shape[0])
        df = preprocess_dataframe(df)
        df = df.fillna("")
        dicts_to_json = []
        for row in df.iterrows():
            curr_dict = dict([])
            curr_dict['context'] = row[1]['Patient Demographic'] + " " + row[1]['Mental Status Exam'] + " " + row[1]['Presenting Problem'] + " " + row[1]['Other Contexts']
            curr_dict['hypothesis'] = row[1]['raw_pred']
            curr_dict['reference'] = row[1]['Explanation for correct answer']
            dicts_to_json.append(curr_dict)
        dfx = pd.DataFrame(dicts_to_json)
        dfx.to_json(f'../generated/{model}_counselingqa.json', index=False, lines=True, orient="records")

1612


/tmp/ipykernel_2480001/776435414.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['raw_pred'].fillna("No answer", inplace=True)
/tmp/ipykernel_2480001/776435414.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

5
1612


/tmp/ipykernel_2480001/776435414.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['raw_pred'].fillna("No answer", inplace=True)
/tmp/ipykernel_2480001/776435414.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

1612


...

/tmp/ipykernel_2480001/776435414.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['raw_pred'].fillna("No answer", inplace=True)
/tmp/ipykernel_2480001/776435414.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

1612


/tmp/ipykernel_2480001/776435414.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['raw_pred'].fillna("No answer", inplace=True)
/tmp/ipykernel_2480001/776435414.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

1612


/tmp/ipykernel_2480001/776435414.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['raw_pred'].fillna("No answer", inplace=True)
/tmp/ipykernel_2480001/776435414.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 